In [ ]:
import pypowsybl.network as pn
import pypowsybl.loadflow as lf
import ipywidgets as widgets

In [ ]:
from pypowsybl_jupyter import NetworkMapWidget

In [ ]:
# Example 1
# load a CGMES file containing a GL profile (Graphical Layout) and run a LF
network1 = pn.load('./data/MicroGridTestConfiguration_T4_BE_BB_Complete_v2.zip', {'iidm.import.cgmes.post-processors': 'cgmesGLImport'})
network_lf_results=lf.run_ac(network1)

In [ ]:
#declare a text place where events data will be displayed
out_events = widgets.Output()
def print_infos(data):   
    with out_events:
        print(data)
print_infos('Events log:')

In [ ]:
#demonstrates how-to create on-hover info (currently, only lines are supported)
def format_to_html_table(row, id, type):
    table = (
        row.to_frame()
        .style.set_caption(f"{type}: {id}")
        .set_table_styles(
            [
                {
                    "selector": "caption",
                    "props": "caption-side: top; font-weight: bold; background-color: #f8f8f8; border-bottom: 1px solid #ddd; width: fit-content; white-space: nowrap;",
                },
                {
                    "selector": "th",
                    "props": "text-align: left; font-weight: bold; background-color: #f8f8f8;",
                },
                {
                    "selector": "td",
                    "props": "text-align: left;",
                },
            ]
        )
        .format(precision=3, thousands=".", decimal=",")
        .set_table_attributes('border="0"')
        .hide(axis="columns")
        .to_html()
    )
    return table

def get_equipment_info(id):
    return format_to_html_table(network1.get_lines().loc[id], id, 'LINE')

In [ ]:
#create a map widget for the network and activate a callback on the selected VL event
map_widget1 = NetworkMapWidget(network1, on_hover_func=get_equipment_info)
map_widget1.on_selectvl(lambda event : print_infos('Selected VL : ' + event.selected_vl))

In [ ]:
#finally, display the widget
widgets.HBox([map_widget1, out_events])

In [ ]:
# Example 2
# load a network that includes geographical data extensions, run a LF and display the network with the NetworkMap widget

In [ ]:
network2=pn.load('./data/demo_geo1.xml')
lf_results2=lf.run_ac(network2)

In [ ]:
network_map2=NetworkMapWidget(network2)

In [ ]:
display(network_map2)

In [ ]:
# center the map on a specific substation
network_map2.center_on_substation('P1')

In [ ]:
# Example 3
# creates a network that contains lines and HVDC lines, and adds a substation and a tie line.
# Then, adds some geographical data extensions, not present in the original network (fictitious coordinates).
# Finally, display the network using the mapviewer widget

network3 = pn.create_four_substations_node_breaker_network()

import pandas as pd
network3.create_substations(id='S5')
network3.create_voltage_levels(id='S5VL1', substation_id='S5', topology_kind='BUS_BREAKER', nominal_v=400)
network3.create_buses(id='S5VL1B1', voltage_level_id='S5VL1')
network3.create_dangling_lines(id='d1', voltage_level_id='S4VL1', node=1, p0=10, q0=3, r=0, x=5, g=0, b=1e-6)
network3.create_dangling_lines(id='d2', voltage_level_id='S5VL1', bus_id='S5VL1B1', p0=10, q0=3, r=0, x=5, g=0, b=1e-6)
network3.create_tie_lines(id='t1', dangling_line1_id='d1', dangling_line2_id='d2')
coords = pd.DataFrame(
    [
        ['S1', 46.648820589226624, 2.9653506942899255],
        ['S2', 45.224159481986970, 4.4868965299190675],
        ['S3', 44.090773206380350, 2.4022589283484335],
        ['S4', 44.279779791063575, 6.0803264207747825],
        ['S5', 44.279779791063575, 7.4221621183374900]
    ], columns=['id', 'latitude', 'longitude']
).set_index('id')
network3.create_extensions('substationPosition', coords[['latitude', 'longitude']])

network_map3=NetworkMapWidget(network3)
display(network_map3)